In [ ]:
# clean old PDF data and append to master list
clean_data = []
old_lines = scrape_pdfs(OLD_DISCOVER_PDFS, OLD_DISCOVER_REGEX)

for line in old_lines:
    split_line = line.split()
    split_date = split_line[0:3]
    date = ['/'.join([months_dict[split_date[1]], split_date[2], split_date[0]])]
    merchant = [' '.join(x for x in split_line[5:-2])]
    amount = [split_line[-1:][0].replace(',', '')]
    clean_data.append(date + ['Discover'] + amount + merchant)

In [ ]:
# clean new PDF data and append to master list

new_lines = [line.replace('\n', ' ') if '\n' in line else line for line in new_lines] # replace newline characters
new_lines = [line[:line.index('PREVIOUS')] if 'PREVIOUS' in line else line for line in new_lines] # delete PREVIOUS in lines

for line in new_lines:
    split_line = line.split()
    date = ['/'.join([split_line[1].replace('0', ''), split_line[0]]) if split_line[1].startswith('0') else '/'.join([split_line[1], split_line[0]])]
    merchant = [' '.join(x for x in split_line[2:-1])]
    amount = [split_line[-1].replace('$', '')]
    clean_data.append(date + ['Discover'] + amount + merchant)


In [ ]:
data_types = {
    'Account': object,
    'Amount': float,
    'Recipient': object,
    'Category': object,
    'SubCategory': object,
    'Project': object,
    'Note': object
}

In [ ]:
df = pd.DataFrame(clean_data, columns=['Date', 'Account', 'Amount', 'Recipient'])
df['Category'] = np.NaN
df['SubCategory'] = np.NaN
df['Project'] = np.NaN
df['Note'] = np.NaN

df = df.astype(data_types)
df['Amount'].apply(lambda x: x * -1).sum()
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index, format="%m/%d/%Y", exact=True)
df.sort_index(ascending=True, inplace=True)
# df.to_excel('discover_transactions.xlsx')

In [ ]:
df.head(20)